In [1]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing SARIMA Model - RMSE per forecast day")
print("=" * 60)

# SARIMA parameters (example: adjust as needed)
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 24)  # 24 for hourly data with daily seasonality

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df['target_datetime'] <= run_date_utc]
        test_data = df[df['target_datetime'] > run_date_utc]

        # Drop any missing data in training
        train_data = train_data.dropna(subset=['target_datetime', target])

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Prepare data for SARIMA
        # Zorg dat index datetime64[ns] en zonder tz is
        train_data['target_datetime'] = pd.to_datetime(train_data['target_datetime']).dt.tz_localize(None)
        test_data['target_datetime'] = pd.to_datetime(test_data['target_datetime']).dt.tz_localize(None)
        y_train = train_data.set_index('target_datetime')[target].astype(float)
        y_test = test_data.set_index('target_datetime')[target].astype(float)

        # Fit SARIMA model
        model = SARIMAX(y_train, order=order, seasonal_order=seasonal_order, enforce_stationarity=False, enforce_invertibility=False)
        model_fit = model.fit(disp=False)

        # Forecast for the test period
        n_test = len(y_test)
        forecast = model_fit.forecast(steps=n_test)
        y_pred = forecast.values

        # Sla de eerste 24 uur over
        if len(y_pred) > 24:
            y_pred = y_pred[24:]
            y_test = y_test[24:]
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan
            rmse_results.append({
                'iteration': i + 1,
                'run_date': run_date.strftime('%Y-%m-%d'),
                'valid_predictions': 0,
                'rmse': rmse
            })
            continue

        if len(y_pred) > 0:
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            rmse = np.nan

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(y_pred),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(y_pred)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - SARIMA Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-05-31 20:47:23,631 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-05-31 20:47:23,631 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-31 20:47:23,632 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)
2025-05-31 20:47:23,632 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-05-31 20:47:23,632 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-31 20:47:23,633 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-31 20:47:23,634 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

🔍 Testing SARIMA Model - RMSE per forecast day


2025-05-31 20:47:34,636 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-16 00:00:00+00:00 for lagging support
2025-05-31 20:47:34,637 - build_training_set - INFO - 🚀 Start build van trainingset
2025-05-31 20:47:34,638 - build_training_set - INFO - 🧠 Actuals van 2025-01-02 00:00:00+00:00 t/m 2025-03-15 23:00:00+00:00 (extended to 2025-03-16 00:00:00+00:00 for lagging)
2025-05-31 20:47:34,639 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-16 00:00:00+00:00, normalized to 2025-03-16 00:00:00+00:00 for DB lookup, target range: 2025-03-16 00:00:00+00:00 → 2025-03-23 00:00:00+00:00
2025-05-31 20:47:34,640 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-05-31 20:47:34,642 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-05-31 20:47:34,643 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'di

Day 1: ✅ 144 test rows, Run: 03-15


2025-05-31 20:47:34,844 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-05-31 20:47:34,845 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-05-31 20:47:34,847 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-05-31 20:47:34,848 - build_training_set - INFO - ✅ No overlap between actuals and predictions
2025-05-31 20:47:34,850 - build_training_set - INFO - 📦 Final combined table: 1921 rows, 21 columns
2025-05-31 20:47:34,851 - build_training_set - INFO - 🧾 Final columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos']
2025-05-31 20:47:34,852 - build_training_set - INFO - 📅 Date range: 2025-01-02 00:00:00+00:00 →

...

Day 27: ✅ 144 test rows, Run: 04-10


2025-06-01 03:33:07,207 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-06-01 03:33:07,208 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-06-01 03:33:07,208 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-06-01 03:33:07,210 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-06-01 03:33:07,222 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-06-01 03:33:07,222 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-06-01 03:33:07,223 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-06-01 03:33:07,225 - build_training_set - INFO - ✅ No overlap between actuals and predictions
2025-06-01 03:33:07,226 - build_training_set - INFO - 📦 Final combined table: 1921 rows, 21 columns
2025-06-01 03:33:07,226 - build_training_set - INFO - 🧾 Final columns: ['Price', 'target_dateti

Day 28: ✅ 144 test rows, Run: 04-11


2025-06-01 03:33:19,150 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-06-01 03:33:19,151 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-06-01 03:33:19,152 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-06-01 03:33:19,154 - build_training_set - INFO - ✅ No overlap between actuals and predictions
2025-06-01 03:33:19,155 - build_training_set - INFO - 📦 Final combined table: 1921 rows, 21 columns
2025-06-01 03:33:19,155 - build_training_set - INFO - 🧾 Final columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos']
2025-06-01 03:33:19,155 - build_training_set - INFO - 📅 Date range: 2025-01-29 00:00:00+00:00 →

Day 29: ✅ 144 test rows, Run: 04-12
Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - SARIMA Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.05
         2 2025-03-16                144  0.04
         3 2025-03-17                144  0.06
         4 2025-03-18                144  0.04
         5 2025-03-19                144  0.04
         6 2025-03-20                144  0.05
         7 2025-03-21                144  0.04
         8 2025-03-22                144  0.12
         9 2025-03-23                144  0.02
        10 2025-03-24                144  0.05
        11 2025-03-25                144  0.04
        12 2025-03-26                144  0.05
        13 2025-03-27                144  0.05
        14 2025-03-28                144  0.04
        15 2025-03-29                144  0.06
        16 2025-03-30                144  0.11
        17 2025-03-31                144  0.04
        18 2025-04-01       

In [2]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
import sys
from pathlib import Path
import warnings

warnings.filterwarnings("ignore")

# Zorg dat build_training_set geïmporteerd is
current_dir = Path.cwd()
while current_dir.name != "ENEXIS" and current_dir.parent != current_dir:
    current_dir = current_dir.parent
project_root = current_dir
utils_path = project_root / "src" / "utils"
sys.path.append(str(utils_path))
from build_training_set import build_training_set

FEATURES = [
    'Load', 'shortwave_radiation', 'temperature_2m', 
    'direct_normal_irradiance', 'diffuse_radiation', 
    'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 
    'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 
    'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 
    'hour_sin', 'weekday_cos'
]
target = 'Price'

# Initial training window
base_start = "2025-01-01 00:00:00"
base_end = "2025-03-14 23:00:00"
base_run = "2025-03-15 00:00:00"

rmse_results = []

print("🔍 Testing SARIMAX Model - RMSE per forecast day")
print("=" * 60)

# SARIMAX parameters (example: adjust as needed)
order = (1, 1, 1)
seasonal_order = (1, 1, 1, 24)  # 24 for hourly data with daily seasonality

for i in range(30):
    start = pd.Timestamp(base_start) + pd.Timedelta(days=i)
    end = pd.Timestamp(base_end) + pd.Timedelta(days=i)
    run_date = pd.Timestamp(base_run) + pd.Timedelta(days=i)

    try:
        df = build_training_set(
            train_start=start.strftime("%Y-%m-%d %H:%M:%S"),
            train_end=end.strftime("%Y-%m-%d %H:%M:%S"),
            run_date=run_date.strftime("%Y-%m-%d %H:%M:%S")
        )

        if df is None or df.empty:
            print(f"Day {i+1}: ❌ No training data returned")
            continue

        df['target_datetime'] = pd.to_datetime(df['target_datetime'], utc=True)
        df = df.sort_values('target_datetime')

        run_date_utc = run_date.tz_localize("UTC")

        # Split into training and testing sets
        train_data = df[df['target_datetime'] <= run_date_utc]
        test_data = df[df['target_datetime'] > run_date_utc]

        # Drop any missing data in training and test
        train_data = train_data.dropna(subset=['target_datetime', target] + FEATURES)
        test_data = test_data.dropna(subset=['target_datetime', target] + FEATURES)

        if test_data.empty or train_data.empty:
            print(f"Day {i+1}: ❌ Not enough data for training or testing")
            continue

        # Prepare data for SARIMAX
        train_data['target_datetime'] = pd.to_datetime(train_data['target_datetime']).dt.tz_localize(None)
        test_data['target_datetime'] = pd.to_datetime(test_data['target_datetime']).dt.tz_localize(None)
        y_train = train_data.set_index('target_datetime')[target].astype(float)
        y_test = test_data.set_index('target_datetime')[target].astype(float)
        exog_train = train_data.set_index('target_datetime')[FEATURES].astype(float)
        exog_test = test_data.set_index('target_datetime')[FEATURES].astype(float)

        # Fit SARIMAX model with exogenous variables
        model = SARIMAX(
            y_train,
            exog=exog_train,
            order=order,
            seasonal_order=seasonal_order,
            enforce_stationarity=False,
            enforce_invertibility=False
        )
        model_fit = model.fit(disp=False)

        # Forecast for the test period
        n_test = len(y_test)
        forecast = model_fit.forecast(steps=n_test, exog=exog_test)
        y_pred = forecast.values

        # Sla de eerste 24 uur over
        if len(y_pred) > 24:
            y_pred = y_pred[24:]
            y_test = y_test[24:]
        else:
            print("Niet genoeg testdata na lag van 24 uur.")
            rmse = np.nan
            rmse_results.append({
                'iteration': i + 1,
                'run_date': run_date.strftime('%Y-%m-%d'),
                'valid_predictions': 0,
                'rmse': rmse
            })
            continue

        if len(y_pred) > 0:
            rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        else:
            rmse = np.nan

        print(f"RSME is {rmse:.4f} for day {i+1}")

        rmse_results.append({
            'iteration': i + 1,
            'run_date': run_date.strftime('%Y-%m-%d'),
            'valid_predictions': len(y_pred),
            'rmse': rmse
        })

        print(f"Day {i+1}: ✅ {len(y_pred)} test rows, Run: {run_date.strftime('%m-%d')}")

    except Exception as e:
        print(f"Day {i+1}: ❌ Error: {e}")

# Create results dataframe
if rmse_results:
    rmse_df = pd.DataFrame(rmse_results)

    print(f"\n📊 OVERALL RMSE - SARIMAX Model")
    print("=" * 80)
    print(f"Successful runs: {rmse_df['rmse'].notna().sum()}/30")

    print(rmse_df[['iteration', 'run_date', 'valid_predictions', 'rmse']].round(2).to_string(index=False))

    print(f"\n📈 SUMMARY STATISTICS")
    print("-" * 40)
    print(rmse_df['rmse'].describe().round(2))

    print(f"\n📊 AVERAGE OVERALL RMSE")
    print("-" * 40)
    print(f"Mean RMSE: {rmse_df['rmse'].mean():.4f}")
    print(f"Stddev RMSE: {rmse_df['rmse'].std():.4f}")

else:
    print("❌ No runs completed successfully")

2025-06-01 04:37:54,343 - build_training_set - INFO - 📅 Loading additional historical data until 2025-03-15 00:00:00+00:00 for lagging support
2025-06-01 04:37:54,357 - build_training_set - INFO - 🚀 Start build van trainingset
2025-06-01 04:37:54,359 - build_training_set - INFO - 🧠 Actuals van 2025-01-01 00:00:00+00:00 t/m 2025-03-14 23:00:00+00:00 (extended to 2025-03-15 00:00:00+00:00 for lagging)


🔍 Testing SARIMAX Model - RMSE per forecast day


2025-06-01 04:37:54,373 - build_training_set - INFO - 📅 Forecast van run_date 2025-03-15 00:00:00+00:00, normalized to 2025-03-15 00:00:00+00:00 for DB lookup, target range: 2025-03-15 00:00:00+00:00 → 2025-03-22 00:00:00+00:00
2025-06-01 04:53:57,871 - build_training_set - INFO - 📥 Loading actuals with selected columns only...
2025-06-01 04:53:57,873 - build_training_set - INFO - 📋 Requested columns found: 21/21
2025-06-01 04:53:57,874 - build_training_set - INFO - 📋 Using columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos']
2025-06-01 04:53:57,889 - build_training_set - INFO - ✅ Actuals loaded: 1752 rows with 21 selected columns
2025-06-01 04:53:57,890 - build_training_set - INFO - 🔍 Loading forecast/prediction 

RSME is 0.0759 for day 1
Day 1: ✅ 144 test rows, Run: 03-15


2025-06-01 08:00:30,003 - build_training_set - INFO -    ✅ Added Flow_NO: 169/169 values found
2025-06-01 08:00:30,004 - build_training_set - INFO -    🕐 Lagging column 'Flow_GB' by 168 hours
2025-06-01 08:00:30,031 - build_training_set - INFO -    ✅ Added Flow_GB: 169/169 values found
2025-06-01 08:00:30,032 - build_training_set - INFO - 🔄 Combining actuals and predictions...
2025-06-01 08:00:30,032 - build_training_set - INFO - 💰 Retrieving actual prices for forecast period...
2025-06-01 08:00:30,034 - build_training_set - INFO - 📊 Found 169 actual prices for forecast period
2025-06-01 08:00:30,047 - build_training_set - INFO - ✅ Filled 169/169 prediction prices with actual values
2025-06-01 08:00:30,047 - build_training_set - INFO - 💰 Price coverage: 169/169 (100.0%)
2025-06-01 08:00:30,049 - build_training_set - INFO - ✅ Combined dataset: 1921 rows (1752 actuals + 169 predictions)
2025-06-01 08:00:30,051 - build_training_set - INFO - ✅ No overlap between actuals and predictions
202

...

RSME is 0.0286 for day 28
Day 28: ✅ 144 test rows, Run: 04-11


2025-06-01 09:56:48,290 - build_training_set - INFO - ✅ No overlap between actuals and predictions
2025-06-01 09:56:48,291 - build_training_set - INFO - 📦 Final combined table: 1921 rows, 21 columns
2025-06-01 09:56:48,291 - build_training_set - INFO - 🧾 Final columns: ['Price', 'target_datetime', 'Load', 'shortwave_radiation', 'temperature_2m', 'direct_normal_irradiance', 'diffuse_radiation', 'Flow_NO', 'yearday_cos', 'Flow_GB', 'month', 'is_dst', 'yearday_sin', 'wind_speed_10m', 'is_non_working_day', 'hour_cos', 'is_weekend', 'cloud_cover', 'weekday_sin', 'hour_sin', 'weekday_cos']
2025-06-01 09:56:48,291 - build_training_set - INFO - 📅 Date range: 2025-01-29 00:00:00+00:00 → 2025-04-19 00:00:00+00:00
2025-06-01 09:56:48,292 - build_training_set - INFO - 💰 Price NaN count: 0/1921 (0.0%)
2025-06-01 09:56:48,293 - build_training_set - INFO - ✅ All columns have good data quality (<20% NaN)
2025-06-01 09:56:48,300 - build_training_set - INFO - ✅ Saved as training_set in WARP.db
2025-06-0

RSME is 0.0326 for day 29
Day 29: ✅ 144 test rows, Run: 04-12


2025-06-01 09:57:21,425 - build_training_set - INFO - 💰 Price NaN count: 0/1921 (0.0%)
2025-06-01 09:57:21,429 - build_training_set - INFO - ✅ All columns have good data quality (<20% NaN)
2025-06-01 09:57:21,440 - build_training_set - INFO - ✅ Saved as training_set in WARP.db
2025-06-01 09:57:21,441 - build_training_set - INFO - 🔒 Connection closed


RSME is 0.0641 for day 30
Day 30: ✅ 144 test rows, Run: 04-13

📊 OVERALL RMSE - SARIMAX Model
Successful runs: 30/30
 iteration   run_date  valid_predictions  rmse
         1 2025-03-15                144  0.08
         2 2025-03-16                144  0.05
         3 2025-03-17                144  0.07
         4 2025-03-18                144  0.04
         5 2025-03-19                144  0.04
         6 2025-03-20                144  0.04
         7 2025-03-21                144  0.05
         8 2025-03-22                144  0.22
         9 2025-03-23                144  0.03
        10 2025-03-24                144  0.07
        11 2025-03-25                144  0.04
        12 2025-03-26                144  0.04
        13 2025-03-27                144  0.06
        14 2025-03-28                144  0.06
        15 2025-03-29                144  0.08
        16 2025-03-30                144  0.21
        17 2025-03-31                144  0.06
        18 2025-04-01                